In [1]:
#Kompresja LZW

#jednocześnie tworzenie słownika i kodowanie

In [2]:
from random import randint
import random as random
from math import log
import bitarray as ba
import operator

In [3]:
def read_file(file):
    return open(file).read()

In [4]:
#słownik dla liczb od 0 - 255
def generateDictionary():
    dict = {}
    for i in range(256):
        dict[bin(i)[2:].zfill(8)] = i
    
    return dict

In [5]:
def read_file_bin(file_name):
    bit_array = ba.bitarray()
    with open(file_name, 'rb') as fh:
        bit_array.fromfile(fh)
    return bit_array

In [6]:
def save_file_bin(file_name, bit_array):
    with open(file_name, 'wb') as fh:
        bit_array.tofile(fh)
    return 1

In [7]:
wikipedia = read_file_bin("norm_wiki_sample.txt")

In [167]:
test = read_file_bin("test.txt")

In [220]:
#Koduje tekst
def encode(text):
    dictionary = generateDictionary()
    
    numbers = []
    max_dict_val = max(dictionary.values())
    
    
    
    print("max_dict_val", max_dict_val)
    
    coded_text = ba.bitarray()
    
    p = text[0:8].to01()
    print("p: ", p)
    
    for i in range(8, len(text), 8):
        #c - obecny ciąg binarny
        c = text[i:i+8].to01()
        if p + c in dictionary:
            #kod znajduje się w słowniku
            p = p + c
        else:
            #kod nie znajduje się w słowniku
            numbers.append(dictionary[p])
            max_dict_val += 1
            dictionary[p + c] = max_dict_val                        
            p = c
            
    numbers.append(dictionary[c]) 
        
    max_number = max(numbers)
    max_length = len(bin(max_number)[2:])
    print("max_num", max_number)
    print("bin(max_number)", bin(max_number)[2:])
    print("max_length", max_length)
    
    
    dictionary = dict((v,k) for k,v in dictionary.items())

    
    
    print("length numbers:", len(numbers))
    for number in numbers:
        for b in bin(number)[2:].zfill(max_length):
            if b == '1':
                coded_text.append(True)
            else:
                coded_text.append(False) 
  
    return coded_text, max_length

In [10]:
#Zapis kodu i zakodowanego tekstu do pliku
def save(text, file_name):
    
    save_file_bin(text, file_name)  
    
    return 0

In [11]:
import struct

In [253]:
#Dekoduje tekst
def decode(coded_text, code_len):
    
    dictionary = generateDictionary()
    
    #swap key and value
    dictionary = dict((v,k) for k,v in dictionary.items())
    
    max_dict_val = max(dictionary.keys())
    print("max_dict_val", max_dict_val)
    
    encoded_text = ba.bitarray()    
    
    #pierwszy element dodany do słownika
    p = coded_text[0:code_len].to01()
    p = int(p, 2)
    print("p: ", p)
    max_dict_val += 1 
    dictionary[max_dict_val] = dictionary[p]
    
    
    for b in dictionary[p]:      
        print("b:", b)
        if b == '1':
            encoded_text.append(True)
        else:
            encoded_text.append(False) 
    
    print(dictionary[p])
    
    
    progress = 0
    step = 1
    
    t = 0

    print("len(coded_text)", len(coded_text))
    print("len(coded_text)/code_len", len(coded_text)/code_len)
    length = (len(coded_text) // code_len) * code_len
    print("length:", length)
    for i in range(code_len, length, code_len):
        #c - obecny ciąg binarny
        c = coded_text[i:i+code_len].to01()
        c = int(c, 2) 

        dictionary[max_dict_val] = dictionary[max_dict_val] + dictionary[c][:8]
        
        t += 1
        for b in dictionary[c]:
            if b == '1':
                encoded_text.append(True)
            else:
                encoded_text.append(False) 
        
        max_dict_val += 1 
        dictionary[max_dict_val] = dictionary[c]
        
        progress += code_len
        if(progress > (len(coded_text) / 100) * step):
            #print(step, "%")
            step += 1
            
    print("processed numbers:", t)
    print("done")
    return encoded_text


In [235]:
def save_file(file_name):
    result = open(file_name, 'w')
    
    for text, num in code.items():
        result.write(text + ";" + str(num) + ";")
    
    result.close()
    return result

In [236]:
#Wczytuje plik binarny
def load(file_name):
    
    file = read_file_bin(file_name)
    
    return file

In [237]:
def compare_strings(original, new_string):
    for org, new in zip(original, new_string):
        if org != new:
            print(org, "!=", new)
            return "Teksty są różne"
    return "Teksty sa równe"

In [238]:
lena = load("lena.bmp")

In [257]:
coded_lena, code_len = encode(lena)

max_dict_val 255
p:  01000010
max_num 2683602
bin(max_number) 1010001111001011010010
max_length 22
length numbers: 2683910


In [240]:
save("coded_lena", coded_lena)

0

In [241]:
coded_lena = load("coded_lena")

In [258]:
decoded_lena = decode(coded_lena, code_len)

max_dict_val 255
p:  66
b: 0
b: 1
b: 0
b: 0
b: 0
b: 0
b: 1
b: 0
01000010
len(coded_text) 59046020
len(coded_text)/code_len 2683910.0
length: 59046020
processed numbers: 2683909
done


In [261]:
len(decoded_lena)

92199472

In [262]:
len(lena)

92199504

In [263]:
save("lena2.bmp", decoded_lena)

0

In [264]:
compare_strings(decoded_lena, lena)

True != False


'Teksty są różne'

In [211]:
decoded_lena

bitarray('01000010010000010100001001000001010000010100001001000001010000010100000101000010010000010100001001000001010000100100000101010011')

In [212]:
test

bitarray('01000010010000010100001001000001010000010100001001000001010000010100000101000010010000010100001001000001010000100100000101010011')

In [27]:
decoded = decode(coded, code)

NameError: name 'coded' is not defined

In [38]:
compare_strings(wikipedia, decoded)

'Teksty sa równe'

In [40]:
length = 0
i = 0

for char, c in code.items():
    i += chars[char]
    length += len(c) * chars[char]
    

oczekiwana_dlugosc_kodu = length / i

In [41]:
oczekiwana_dlugosc_kodu

4.3090155002237935

In [42]:
efektywnosc_kodowania2 = entropia_wikipedii / oczekiwana_dlugosc_kodu

In [43]:
efektywnosc_kodowania2

0.9933582848516693